In [1]:
import pandas as pd
import altair as alt
import numpy as np
alt.data_transformers.enable("json")
import matplotlib.pyplot as plt
import scipy

DataTransformerRegistry.enable('json')

# User inputs

In [2]:
start_date = '20221130'
end_date = '20230509'

tidy_dataset_fn = f"../sos/tidy_df_30Min_{start_date}_{end_date}_noplanar_fit.parquet"
tidy_dataset_5min_fn = f"../sos/tidy_df_{start_date}_{end_date}_noplanar_fit.parquet"
tidy_daily_dataset_output_fn = f"tidy_df_daily_{start_date}_{end_date}_noplanar_fit.parquet"

# Load data

In [3]:
try:
    tidy_df_30Min = pd.read_parquet(
        tidy_dataset_fn
    )
except FileNotFoundError:
    print("No file such file exists for these dates.")
tidy_df_30Min['time'] = pd.to_datetime(tidy_df_30Min['time'])

try:
    tidy_df_5Min = pd.read_parquet(
        tidy_dataset_5min_fn
    )
except FileNotFoundError:
    print("No file such file exists for these dates.")
tidy_df_5Min['time'] = pd.to_datetime(tidy_df_5Min['time'])

# Examine pot. virtual temperature gradient

In [4]:
np.abs(tidy_df_30Min[tidy_df_30Min.measurement == 'temperature gradient'].value).min()

1.4920204573882438e-06

In [5]:
alt.Chart(
    tidy_df_30Min.query("measurement == 'temperature gradient'").query("height < 5")
).mark_line().encode(
    x = 'time:T',
    y = 'value:Q',
    column='height:O'
).properties(width=200, height = 200) & alt.Chart(
    tidy_df_30Min.query("measurement == 'temperature gradient'").query("height < 5")
).mark_bar().encode(
    alt.X('value:Q').bin(step=0.1),
    alt.Y("count():Q"),    
    alt.Column('height:O')
).properties(width=200, height = 200)

alt.VConcatChart(...)

In [6]:
src = tidy_df_30Min.query("variable == 'temp_gradient_3m_c'")
neutral_times = src[src['value'].abs() < 0.01].time

In [7]:
alt.Chart(
    tidy_df_30Min[
        tidy_df_30Min.time.isin(neutral_times.sample(16))
    ].query("measurement == 'wind speed'").query("tower == 'c'")
).mark_line().encode(
    alt.X("value:Q").title("Wind speed (m/s)").sort('-y'),
    alt.Y("height:Q").title("Height (m)"),
    alt.Facet("time:O", columns=8)
).properties(width = 125, height = 125)

alt.Chart(...)

In [8]:
src = tidy_df_30Min[
    tidy_df_30Min.time.isin(neutral_times)
].query("tower == 'c'")
src_snowdepth = tidy_df_30Min[
    tidy_df_30Min.measurement == 'snow depth'
]
src_snowdepth = src_snowdepth[['time', 'value']].set_index('time').rename(columns={'value': 'snow_depth'})
src = src[src.measurement.isin([
    'wind speed',
    'shear velocity',
    'snow depth'
])]
src = src[~src.variable.str.contains("predicted")]
src = src.pivot_table(index=['time'], values='value', columns='variable')
src = src.join(src_snowdepth)
src.head()

,spd_10m_c,spd_15m_c,spd_20m_c,spd_2m_c,spd_3m_c,spd_5m_c,u*_10m_c,u*_15m_c,u*_20m_c,u*_2m_c,u*_3m_c,u*_5m_c,snow_depth
time,,,,,,,,,,,,,
2022-12-02 10:30:00,5.852733,5.990304,6.163234,5.169695,5.503640,5.802035,0.771105,0.711912,0.749025,0.772614,0.748710,0.600726,0.346144
2022-12-02 11:00:00,5.108618,5.132134,5.137251,4.701602,4.848611,5.051483,0.792359,0.733981,0.720212,0.809777,0.727379,0.728357,0.346144
2022-12-02 11:30:00,4.945608,5.086113,5.158172,4.254836,4.424224,4.764127,0.827612,0.762404,0.772660,0.802487,0.794172,0.635123,0.346144
2022-12-02 12:00:00,6.954772,7.209726,7.195507,5.955409,6.454049,6.821709,0.922849,1.007616,1.199078,0.840452,0.820263,0.595426,0.346144
2022-12-02 12:30:00,4.535348,4.960222,4.781742,4.116060,4.270485,4.554066,0.813880,0.840683,0.887759,0.753841,0.708530,0.631443,0.346144


# Filter out data without monotonically increasing wind speeds 

In [9]:
def monotonically_increasing(l):
    return all(x < y for x, y in zip(l, l[1:]))

src['is_monotonic_increasing'] = src.apply(
    lambda row: monotonically_increasing([
            row['spd_2m_c'], 
            row['spd_3m_c'], 
            row['spd_5m_c'], 
            row['spd_10m_c'], 
            row['spd_15m_c'], 
            row['spd_20m_c']
    ]),
    axis = 1
)

In [10]:
src = src[src.is_monotonic_increasing]

# Solve for $z_0$ assuming $d = 0$

https://www.eol.ucar.edu/content/calculation-roughness-length-and-displacement-height

In [11]:
von_karman = 0.4

In [12]:
d = 0.0
src['z0_2m_c'] = (2 - d - src['snow_depth'])/np.exp(src['spd_2m_c']*von_karman/src['u*_2m_c'])
src['z0_3m_c'] = (3 - d - src['snow_depth'])/np.exp(src['spd_3m_c']*von_karman/src['u*_3m_c'])
src['z0_5m_c'] = (5 - d - src['snow_depth'])/np.exp(src['spd_5m_c']*von_karman/src['u*_5m_c'])
src['z0_10m_c'] = (10 - d - src['snow_depth'])/np.exp(src['spd_10m_c']*von_karman/src['u*_10m_c'])
src['z0_15m_c'] = (15 - d - src['snow_depth'])/np.exp(src['spd_15m_c']*von_karman/src['u*_15m_c'])
src['z0_20m_c'] = (20 - d - src['snow_depth'])/np.exp(src['spd_20m_c']*von_karman/src['u*_20m_c'])

In [13]:
variables = [
        'z0_2m_c', 
        'z0_3m_c', 
        'z0_5m_c', 
        'z0_10m_c', 
        'z0_15m_c', 
        'z0_20m_c', 
    ]
alt.Chart(
    src[variables].reset_index()
).transform_fold(
    variables
).transform_filter(
    alt.FieldOneOfPredicate('key', ['z0_2m_c', 'z0_3m_c'])
).mark_circle().encode(
    alt.X('time:T'),
    alt.Y("value:Q").scale(type='log'),
    alt.Row("key:N", sort=variables)
).properties(height = 100, width = 400)

alt.Chart(...)

In [14]:
alt.Chart(
    src[['z0_3m_c']].resample("1D").median().reset_index()
).mark_circle(size=50).encode(
    alt.X("time:T"),
    alt.Y("z0_3m_c").scale(type='log')
) + alt.Chart(
    src[['z0_3m_c']].resample("1D").median().reset_index()
).mark_bar(width=1).encode(
    alt.X("time:T"),
    alt.Y("z0_3m_c").scale(type='log')
).properties(height = 100)

alt.LayerChart(...)

In [104]:
basic_z0_values_chart = alt.Chart(
    src[['z0_3m_c']].resample("W-MON").median().reset_index()
).mark_circle(size=100).encode(
    alt.X("time:T"),
    alt.Y("z0_3m_c").scale(type='log')
) + alt.Chart(
    src[['z0_3m_c']].resample("W-MON").median().reset_index()
).mark_bar(width=1).encode(
    alt.X("time:T"),
    alt.Y("z0_3m_c").scale(type='log')
).properties(height = 100)
basic_z0_values_chart

alt.LayerChart(...)

In [16]:
src[['z0_3m_c']].resample("W-MON").median().reset_index()

,time,z0_3m_c
0,2022-12-05,0.177667
1,2022-12-12,0.007912
2,2022-12-19,0.005446
3,2022-12-26,0.052083
4,2023-01-02,0.000998
5,2023-01-09,0.007609
6,2023-01-16,0.001780
7,2023-01-23,0.001265
8,2023-01-30,0.004634
9,2023-02-06,0.002709


# Use Andreas et al. 2010 Method, NOAA/SPLASH (Chris Cox) solution

In [71]:
tdk = 273.15 
# surface temp mean
Tsm = tidy_df_30Min.query("variable == 'Tsurf_c'")['value'].values 
# height of sonic
z_level_n = 3 - tidy_df_30Min.query("variable == 'SnowDepth_d'")['value'].values 
# wt-covariance, vertical flux of the sonic temperature  [deg m/s]
wT_csp = tidy_df_30Min.query("variable == 'w_tc__3m_c'")['value'].values   
wsp = tidy_df_30Min.query("variable == 'spd_3m_c'")['value'].values 
ustar = tidy_df_30Min.query("variable == 'u*_3m_c'")['value'].values 

# Monin-Obukhov stability parameter, z/L:
zeta_level_n = - ((0.4*9.81)/(Tsm+tdk))*(z_level_n*wT_csp/(ustar**3))
# Drag coefficient, Cd:
Cd = ustar**2/wsp**2 #- wu_csp/(wsp**2)

In [77]:
import sys
sys.path.append("../splash/")
import calc_z0
z0_values = calc_z0.calc_z0(z_level_n, Cd, zeta_level_n)

/home/elilouis/sublimationofsnow/analysis/paper1/../splash/calc_z0.py:16: RuntimeWarning: invalid value encountered in power
  sma = 1 + (6.5 * zL * (1+zL)**(1/3)) / (1.3 + zL); # Psi
/home/elilouis/sublimationofsnow/analysis/paper1/../splash/calc_z0.py:20: RuntimeWarning: invalid value encountered in power
  x = np.real((1 - 16*zL)**(0.25)) # assumes gamma = 16


In [89]:
time_values = tidy_df_30Min.query("variable == 'Tsurf_c'").set_index('time').sort_index().index.values

In [98]:
z0_df = pd.DataFrame({
    "time": time_values, 
    "z0":   z0_values
})
weekly_z0_df = df.set_index('time').resample('W-MON').median().reset_index()

In [117]:
complex_z0_values_chart = alt.Chart(
    weekly_z0_df
).mark_circle(size=100, color='red', opacity=0.5).encode(
    alt.X("time:T"),
    alt.Y("z0").scale(type='log')
) + alt.Chart(
    weekly_z0_df
).mark_bar(width=1).encode(
    alt.X("time:T"),
    alt.Y("z0").scale(type='log')
).properties(height = 100)

In [119]:
(   
    basic_z0_values_chart.properties(title='Basic method') 
    | 
    complex_z0_values_chart.properties(title='Complex method')   
).resolve_scale(y='shared')

alt.HConcatChart(...)

In [118]:
(   
    basic_z0_values_chart.properties(title='Basic method') 
    + 
    complex_z0_values_chart.properties(title='Complex method')   
).resolve_scale(y='shared')

alt.LayerChart(...)

In [ ]:
(   
    basic_z0_values_chart.properties(title='Basic method') 
    + 
    complex_z0_values_chart.properties(title='Complex method')   
).resolve_scale(y='shared')

alt.LayerChart(...)